In [1]:
import findspark
findspark.init()

In [2]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

In [3]:
conf = SparkConf().setAppName('travel_operator_booking').setMaster('local')
sc = SparkContext(conf=conf)
spark = SparkSession(sc)

In [4]:
spark.sql("set spark.sql.legacy.timeParserPolicy=LEGACY")

DataFrame[key: string, value: string]

In [5]:
hotels_data_df =spark.read.format('csv').option('header','true').load('./hotel_bookings.csv')

In [6]:
hotels_data_df.select('market_segment').distinct().show()

+--------------+
|market_segment|
+--------------+
| Offline TA/TO|
| Complementary|
|      Aviation|
|        Direct|
|     Undefined|
|     Corporate|
|     Online TA|
|        Groups|
+--------------+



In [7]:
hotels_to_data_df = hotels_data_df.where('market_segment = "Offline TA/TO"')

In [8]:
hotels_data_transformed_df = hotels_to_data_df\
        .withColumn('arrival_date',F.to_date(F.concat_ws("-",F.col("arrival_date_year"),F.col("arrival_date_month"),F.col("arrival_date_day_of_month")),"yyyy-MMM-dd"))\
        .withColumn('departure_date',F.expr("date_add(arrival_date, cast(stays_in_weekend_nights as int)+cast(stays_in_week_nights as int))"))\
        .withColumn('no_of_children', F.col('children')+F.col('babies'))\
        .withColumn('with_family_breakfast', F.when(F.expr('no_of_children > 0'), F.lit('YES')).otherwise(F.lit('NO')))\
        .drop('no_of_children')

In [10]:
hotels_data_transformed_df.write.mode('overwrite').parquet('./travel_operator_booking')